# INICIALIZAÇÃO

## Instalando as bibliotecas

In [ ]:
# Instalar o Ultralytics
!pip install ultralytics

# Importar a biblioteca
from ultralytics import YOLO

## Montando o drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Salvando os arquivo YAML's de treinamento + validação e teste

In [ ]:
import os

# Definir o caminho da pasta onde o arquivo YAML será salvo
pasta_destino = "pasta/destino"
os.makedirs(pasta_destino, exist_ok=True)  # Criar a pasta se não existir

# Caminhos do dataset (ajuste conforme necessário)
train_path = "pasta/imagens/treino/"
val_path   = "pasta/imagens/val/"
test_path  = "pasta/imagens/test/"

# Conteúdo do arquivo YAML
yaml_content = f"""train: '{train_path}'
val: '{val_path}'

nc: 1  # Número de classes
names: ['obj']
"""

# Caminho completo do arquivo YAML
yaml_train_file = os.path.join(pasta_destino, "dataset_train.yaml")

# Escrever o conteúdo no arquivo
with open(yaml_train_file, "w") as file:
    file.write(yaml_content)

print(f"Arquivo YAML de treino gerado em: {yaml_train_file}")

#-----------------------------------------------------------

# Conteúdo do arquivo YAML
yaml_content = f"""train: '{train_path}'
val: '{test_path}'

nc: 1  # Número de classes
names: ['obj']
"""

# Caminho completo do arquivo YAML
yaml_test_file = os.path.join(pasta_destino, "dataset_test.yaml")

# Escrever o conteúdo no arquivo
with open(yaml_test_file, "w") as file:
    file.write(yaml_content)

print(f"Arquivo YAML de teste gerado em: {yaml_test_file}")


# TREINAMENTO

## Resetando a GPU

In [ ]:
import torch
torch.cuda.empty_cache()


## Descobrindo o número de núcleos disponíveis

In [ ]:
import multiprocessing

workers_max = multiprocessing.cpu_count()
print(f"Número de núcleos disponíveis: {workers_max}")

Número de núcleos disponíveis: 12


## Treinamento yolo

In [ ]:
# Carregar o modelo YOLOv11x
model = YOLO('yolo11x.pt')

results = model.train(
    data=    'pasta/do/dataset_train.yaml',
    project= 'pasta/onde/serao/salvos/resultados',
    name=    'nomedoprojeto',
    imgsz=320,
    workers=workers_max - 2,

    iou=0.7,
    conf=0.01,
    patience=50,   # Reduzir a paciência
    epochs=150,    # Reduzir o número de épocas
    batch=64,      # Aumentar o batch size se possível
    max_det=8,
    seed = 5,

    single_cls	=	False,	#Trata todas as classes em conjuntos de dados multi-classe como uma única classe durante o treino.
    plots = True, #Imprime os resultados

    flipud = 0.5,
    mosaic=0,    # Reativar mosaic augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.1,
    device='cuda', #Treinamento com uso de GPU CUDA

    nms=True,
    save_period=True,
    verbose=True,
    scale=0,#0.2,       # Reativar escala com valor moderado
    shear=0,#0.1,       # Reativar cisalhamento com valor moderado
    copy_paste=0,#0.1,  # Reativar copy-paste com valor moderado

    weight_decay=0.0001, # Reativar weight decay
)

# MÉTRICAS

## Tirando as métricas da base de validação

In [ ]:
# Caminho para o modelo treinado
model_path = r'pasta/weights/seumodelo.pt'

# Carregar o modelo
model = YOLO(model_path)

# Avaliar o modelo usando o arquivo data.yaml
metrics = model.val(data=yaml_train_file, imgsz=320)

## Tirando as métricas da base de teste

In [ ]:
# Caminho para o modelo treinado
model_path = r'pasta/weights/seumodelo.pt'

yaml_test_file = r'pasta/dataset_test.yaml'
# Carregar o modelo
model = YOLO(model_path)

# Avaliar o modelo usando o arquivo data.yaml
metrics = model.val(data=yaml_test_file, imgsz=320)#, conf = 0.25, iou=0.5)